# Ayudantía: Simulación



### Simulación Síncrona

Una forma general de verlo sería:

MIENTRAS el tiempo simulación no termine:
    - SI ocurren eventos en este intervalo de tiempo:
        - simular los eventos
    - Aumentar tiempo en una unidad 

Problemas:

- Ejecución lenta
- Mayoría de incrementos no cambia el estado del sistema
- Las verificaciones generan pérdida de tiempo en la CPU

### Simulación DES

Una forma general de verlo sería:

MIENTRAS la lista de eventos no esté vacía y el tiempo de simulación no termine:
    - Tomar un evento del principio de la lista de eventos
    - Avanzar tiempo de simulación al tiempo del evento
    - Simular el evento


o bien,


MIENTRAS hayan eventos y el tiempo de simulación no termine:
    - Encontrar próximo evento
    - Avanzar tiempo de simulación al tiempo del evento
    - Simular el evento


### Distribuciones

Utilizaremos la libería random. 

- `random.choice(secuencia)`: Escoge un elemento aleatorio de una secuencia (ej: lista)

- `random.randint(a, b)`: Para ENTEROS que distribuyen uniforme (a y b incluidos)

- `random.uniform(a, b)`: Para valores que distribuyen uniforme entre [a, b)

- `random.randrange(a, b, step)`: Para valores que distribuyen uniforme entre [a, b)

- `random.expovariate(lambd)`: Para eventos que distribuyen exponencial (lambd ocurrencias cada una unidad de tiempo <=> ocurrencias ocurren en promedio cada 1/lambd tiempo)


## Problema

Desde el ministerio de transporte, se te ha pedido que implementes un programa que modele y simule el funcionamiento de sus autopistas, pues quiere ver en qué autopistas se debe licitar nuevas estaciones de servicio y gasolineras. Para esto, te dan las siguientes indicaciones: 
 
>"El programa debe ser capaz de simular el funcionamiento de una ruta que tendrá un largo entre 350 y 500 km (que es la distancia que posee dicha ruta para ser completada) y una probabilidad de que un conductor en auto,  llegue a la ruta distribuye exponencial de tasa lambda (tasa de llegada es una variable aleatoria entre 0.05 y 0.1 vehículos por unidad de tiempo). Cada conductor, puede conducir a una velocidad aleatoria entre entre 1 y 3 km/minuto. Además, cada vehículo gasta entre 0.1 y 0.07 litros por cada kilómetro recorrido. Además de esto, los autos no tienen estanques infinitos, por lo que al momento de ingresar a la ruta, se tiene que su estanque tiene una cantidad entera aleatoria entre 40 y 50 litros. Como no hay estaciones de servicio, un auto puede llegar al final de la ruta, o quedarse en pana en la mitad del trayecto. Finalmente, carabineros realiza control a los autos de manera aleatoria: en promedio cada 15 unidades de tiempo se selecciona un auto en la carretera al azar y se le realiza control durante 5 a 10 unidades de tiempo.  Una vez que la simulación termine, se debe informar cuántos autos se quedaron sin bencina, y cuántos lograron terminar la ruta exitosamente."

Del enunciado se puede extraer la siguiente entidad:
- Auto: posee los siguientes atributos:
    - velocidad: cantidad aleatoria (se asume uniforme) entre 1 y 3 kilómetros por unidad de tiempo.
    - rendimiento: cantidad aleatoria (se asume uniforme) entre 0.1 y 0.07 litros por kilómetro.
    - estanque: cantidad aleatoria (se asume uniforme) entre 40 y 50 litros.
 
 
- Ruta: posee los siguientes atributos:
    - largo_ruta: cantidad aleatoria (se asume uniforme) entre 350 y 500 kilómetros.

Como la ruta es la entidad que contiene a todos las entidades del sistema, la utilizaremos para realizar la simulación, por lo que se le implementará un método `run` para llevarla a cabo. Desde ahora, se le llamará a las rutas `Simulacion`.

Además, se pueden extraer los siguientes eventos:

- Control de carabineros: afecta a un auto al azar. Ocurre cada 3 unidades de tiempo y no nos dicen la distribución por lo que asumimos uniforme entre 10 y 20 unidades de tiempo (el 10 y 20 fueron completamente arbitrarios :P).
- Llegada auto a la autopista: crea un nuevo auto que comienza en el comienzo de la ruta. Ocurre a una tasa aleatoria entre 0.05 y 0.1 vehículos por unidad de tiempo que se define al comienzo de la simulación.

Además, para cada vehículo se tienen los siguientes eventos propios de ellos:
- Pana de vehículo.
- Llegada de Vehículo.

Sin embargo, para esta simulación simple, ambos corresponden a un mismo evento propio de los autos:
- Salida de auto: afecta al auto correspondiente. Ocurre cuando un auto llega al final de la ruta, o bien, llega hasta la distancia máxima que su bencina pudo darle.

In [5]:
from random import expovariate, randint, uniform, choice
import numpy as np

In [6]:
class Evento:
    def __init__(self, nombre, tiempo, auto=None):
        self.nombre = nombre
        self.tiempo = tiempo
        self.auto = auto

In [7]:
class Auto:
    num = 0

    def __init__(self, tiempo_inicio, por_recorrer):
        self.tiempo_inicial = tiempo_inicio
        # Variables aleatorias
        self.rendimiento = uniform(0.1, 0.07)   # litros / kilometros
        self.estanque = randint(40, 50)         # litros
        self.velocidad = randint(1, 3)          # kilometros / minuto
        self.por_recorrer = por_recorrer
        self.tiempo_detenido = 0
        # Identificacion de instancias
        self.llega = (self.estanque / self.rendimiento) > por_recorrer   #  bool
        self.id = Auto.num
        Auto.num += 1

    def distancia_recorrida(self, tiempo):
        return round((tiempo - self.tiempo_inicial - self.tiempo_detenido) * self.velocidad, 1)

    @property
    def siguiente_evento(self):
        # Vemos que pasa primero: auto queda en pana o llega
        if self.llega:
            tiempo = self.tiempo_inicial + self.por_recorrer / self.velocidad
            return Evento('salida_ruta', tiempo, self)
        tiempo = self.tiempo_inicial + (self.estanque / self.rendimiento) / self.velocidad
        return Evento('pana', tiempo, self)

In [1]:

class Simulacion:

    def __init__(self, tiempo_maximo):
        self.tiempo_maximo = tiempo_maximo
        self.tiempo_actual = 0
        self.eventos = list()
        self.autos = list()
        self.autos_pana = 0
        self.autos_terminaron = 0
        self.autos_detenidos = 0
        self.largo_ruta = randint(350, 500)
        self.tasa_llegada_autos = uniform(0.05, 0.1)  #  cada 10 a 20 min sale un auto
        self.metodos = {'nuevo_auto': self.nuevo_auto,
                        'salida_ruta': self.salida_ruta,
                        'pana': self.pana, 'carabineros': self.carabineros}

    def eventos_iniciales(self):
        # asumimos que llega en el tiempo cero el primer auto
        self.eventos.append(Evento('nuevo_auto', 0))
        self.eventos.append(Evento('carabineros', uniform(20, 40)))

    def nuevo_auto(self, *args):  #  no usaremos los args, es solo para llamar la fucion de manera generica.
        # Creamos el auto, añadimos evento salida de la ruta o pana segun corresponda
        auto = Auto(args[0].tiempo, self.largo_ruta)
        print("[NUEVO AUTO]: Auto {} ha iniciado su viaje".format(auto.id))
        self.eventos.append(auto.siguiente_evento)
        # agregamos a la lista de autos
        self.autos.append(auto)
        # Definimos eveto de llegada de proximo auto
        tiempo_proximo_auto = args[0].tiempo + expovariate(self.tasa_llegada_autos)
        self.eventos.append(Evento('nuevo_auto', tiempo_proximo_auto))

    def salida_ruta(self, *args):
        evento = args[0]
        print("[LLEGADA]: Auto {} salió de la ruta".format(evento.auto.id))
        self.autos.remove(evento.auto)
        self.autos_terminaron += 1

    def pana(self, *args):
        evento = args[0]
        print("[PANA en km {}]: Auto {} se quedo en pana".format(
            evento.auto.distancia_recorrida(evento.tiempo), evento.auto.id))
        self.autos.remove(evento.auto)
        self.autos_pana += 1

    def carabineros(self, *args):
        evento = args[0]
        if self.autos:
            auto_detenido = choice(self.autos)
            print("[DETENCION en km {}]: Auto {} se le realizó control de carabineros".format(
                    auto_detenido.distancia_recorrida(evento.tiempo),
                    auto_detenido.id))
            evento_salida_auto = filter(lambda evento: evento.auto == auto_detenido, self.eventos)[0]
            # si detienen a un auto se demorara entre 5 y 15 minutos más en quedar en pana o salir de la ruta
            tiempo_detencion = uniform(5, 10)
            evento_salida_auto.tiempo += tiempo_detencion
            auto_detenido.tiempo_detenido += tiempo_detencion
            self.autos_detenidos += 1
        tiempo_proxima_detencion = self.tiempo_actual + uniform(20, 40)
        self.eventos.append(Evento('carabineros', tiempo_proxima_detencion))

    def run(self):
        self.eventos_iniciales()
        print("\n")
        while self.tiempo_actual <= self.tiempo_maximo and self.eventos:
            self.eventos.sort(key=lambda evento: evento.tiempo)
            evento = self.eventos.pop(0)
            if evento.tiempo < self.tiempo_maximo:
                self.tiempo_actual = evento.tiempo
                self.metodos[evento.nombre](evento)

In [14]:
def estadisticas_globales(n=10, tiempo_maximo=500):
    estadisticas = list()
    for i in range(n):
        s = Simulacion(tiempo_maximo)
        s.run()  #  va a imprimir varios logs
        estadisticas.append(
            {'detenidos': s.autos_detenidos, "pana": s.autos_pana,
             'terminaron': s.autos_terminaron, "largo": s.largo_ruta})

    print("\n")
    for i in range(n):
        print(
            "En Simulacion {} - {} kms: {} autos se quedaron en pana. {} autos llegaron al final de la ruta. {} autos fueron detenidos".format(
                i, estadisticas[i]["largo"], estadisticas[i]["pana"], estadisticas[i]["terminaron"],
                estadisticas[i]["detenidos"]))
    print("\nEstadisticas Promedio:")
    print("   Promedio largo rutas:        {0:<10.6}".format(
        np.mean([est['largo'] for est in estadisticas])))
    print("   Promedio autos en Pana:        {0:<10.6}".format(
        np.mean([est['pana'] for est in estadisticas])))
    print("   Promedio autos en Llegados:    {0:<10.6}".format(
        np.mean([est['terminaron'] for est in estadisticas])))
    print("   Promedio autos en Detenidos:   {0:<10.6}".format(
        np.mean([est['detenidos'] for est in estadisticas])))


In [15]:
if __name__ == '__main__':
    estadisticas_globales(n=10)




[NUEVO AUTO]: Auto 785 ha iniciado su viaje
[NUEVO AUTO]: Auto 786 ha iniciado su viaje
[DETENCION en km 13.8]: Auto 786 se le realizó control de carabineros
[NUEVO AUTO]: Auto 787 ha iniciado su viaje
[DETENCION en km 0.1]: Auto 787 se le realizó control de carabineros
[NUEVO AUTO]: Auto 788 ha iniciado su viaje
[DETENCION en km 60.2]: Auto 786 se le realizó control de carabineros
[NUEVO AUTO]: Auto 789 ha iniciado su viaje
[NUEVO AUTO]: Auto 790 ha iniciado su viaje
[NUEVO AUTO]: Auto 791 ha iniciado su viaje
[DETENCION en km 4.7]: Auto 791 se le realizó control de carabineros
[NUEVO AUTO]: Auto 792 ha iniciado su viaje
[DETENCION en km 76.4]: Auto 790 se le realizó control de carabineros
[NUEVO AUTO]: Auto 793 ha iniciado su viaje
[DETENCION en km 10.1]: Auto 793 se le realizó control de carabineros
[NUEVO AUTO]: Auto 794 ha iniciado su viaje
[NUEVO AUTO]: Auto 795 ha iniciado su viaje
[DETENCION en km 228.6]: Auto 791 se le realizó control de carabineros
[LLEGADA]: Auto 787 salió